<a href="https://colab.research.google.com/github/KevinHern/AI-Crash-Course/blob/main/AI_Crash_Course_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks

[Presentation: AI Crash Course 07](https://view.genial.ly/61a033a3d9c41f0da944cd11/presentation-ai-crashcourse07)

## 0) Preparations

In [2]:
# ----- Libraries ----- #

# For graph plotting
import matplotlib.pyplot as plt
from tensorflow.math import confusion_matrix

# For dataset manipulation
import numpy as np

# For visualizing more complex graphs
import seaborn as sns

# Neural networks
import tensorflow as tf

# Global constant for training acceleration
AUTOTUNE = tf.data.AUTOTUNE

# Lets load an extension that helps us to visualize the performance of our model
%load_ext tensorboard

## 1) Dataset Preparations

In [ ]:
# Lets use an already available dataset
from tensorflow.keras import datasets

# Lets try to classify numbers between 0 and 9
(train_images, __), (test_images, __) = datasets.mnist.load_data()

# A good practice when dealing with images: Normalize images from 0 to 1 or from -1 to 1.
# We do not want our neural net to have big weights → one slight change and the outcome may be completely different!
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
# Lets visualize a few samples
print(train_images[0].shape)
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
plt.show()

# Model

## 2) Encoder Definition

In [5]:
class Encoder(tf.keras.models.Model):
  def __init__(self, latent_dim):
    super(Encoder, self).__init__()
    
    self.inputs = tf.keras.layers.InputLayer(input_shape=(28,28,1))

    ''' Actual Model Architecture '''
    self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu')
    self.batch1 = tf.keras.layers.BatchNormalization()
    self.conv2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu')
    self.batch2 = tf.keras.layers.BatchNormalization()
    self.conv3 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu')
    self.batch3 = tf.keras.layers.BatchNormalization()

    ''' Latent Space '''
    self.flatten = tf.keras.layers.Flatten()
    self.latent_space = tf.keras.layers.Dense(units=latent_dim, activation='relu')


  def call(self, x):
    
    x = self.inputs(x)

    x = self.conv1(x)
    x = self.batch1(x)
    x = self.conv2(x)
    x = self.batch2(x)
    x = self.conv3(x)
    x = self.batch3(x)

    x = self.flatten(x)
    outputs = self.latent_space(x)
    
    return outputs

In [ ]:
dummy_encoder = Encoder(latent_dim=128)
dummy_result = dummy_encoder(np.reshape(train_images[0:10], newshape=(10, 28, 28, 1)))
dummy_result.shape

## 3) Decoder Definition

In [64]:
class Decoder(tf.keras.models.Model):
  def __init__(self, latent_dim):
    super(Decoder, self).__init__()

    self.inputs = tf.keras.layers.InputLayer(input_shape=(latent_dim))
    self.reshape = tf.keras.layers.Reshape(target_shape=(1 , 1, latent_dim))

    ''' Actual Model Architecture '''
    self.ct1 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(2,2), strides=(2,2), activation='relu', padding='same')
    self.batch1 = tf.keras.layers.BatchNormalization()
    
    self.ct2 = tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=(2,2), strides=(2,2), activation='relu', padding='same')
    self.batch2 = tf.keras.layers.BatchNormalization()
    
    self.ct3 = tf.keras.layers.Conv2DTranspose(filters=8, kernel_size=(4,4), strides=(7,7), activation='relu', padding='same')
    self.conv1 = tf.keras.layers.Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same')

  def call(self, x):
    x = self.inputs(x)
    x = self.reshape(x)

    x = self.ct1(x)
    x = self.batch1(x)
    x = self.ct2(x)
    x = self.batch2(x)
    x = self.ct3(x)

    outputs = self.conv1(x)

    return outputs

In [65]:
dummy_decoder = Decoder(latent_dim=128)
results = dummy_decoder(dummy_result)

In [ ]:
plt.figure(figsize=(10,10))

original = True
for i in range(10):
  plt.subplot(10,2,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  if original:
    plt.imshow(np.reshape(train_images[i], newshape=(28, 28)))
    original = False
    i -= 1
  else:
    plt.imshow(np.reshape(results[i], newshape=(28, 28)))
    original = True
plt.show()

## 4) Autoencoder Definition

In [67]:
class Autoencoder(tf.keras.models.Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.encoder = Encoder(latent_dim=latent_dim)
    self.decoder = Decoder(latent_dim=latent_dim)

  def call(self, x):
    x = self.encoder(x)
    outputs = self.decoder(x)
    return outputs

In [ ]:
dummy_autoencoder = Autoencoder(latent_dim=128)
dummy_autoencoder(np.reshape(train_images[0], newshape=(1, 28, 28, 1))).shape

## 5) Callbacks

In [ ]:
'''
  CALLBACKS:
  These are functions that are called once a certain amount of epochs end. These come very handful to deal with overfitting
'''

# Lets load an extension that helps us to visualize the performance of our model
%load_ext tensorboard

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/model/', histogram_freq=1)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('logs/model/', monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_freq='epoch')
earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,)

## 6) Training

In [ ]:
# Lets build the model. NOTE: this is the construction of the architecture of the model!
autoencoder = Autoencoder(latent_dim=128)

# Now lets compile the model. NOTE: These are the finishing touches before having a fully functional model
autoencoder.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=['mse'])

# Before training, lets reshape our training date
train_images = np.reshape(
  train_images,
  newshape=(train_images.shape[0], train_images.shape[1], train_images.shape[2], 1)
)


# Now lets train the model!
autoencoder.fit(train_images,
          train_images,
          epochs=100,
          batch_size = 128,
          validation_split=0.2,
          callbacks=[tensorboard_callback, checkpoint_callback, earlystopping_callback]
        )

# Lets evaluate our model
#model.evaluate(x=test_images, y=test_labels, batch_size=128)

In [ ]:
# Lets visualize
%tensorboard --logdir "logs/model/" --port=8008

## 7) Lets watch the results

In [72]:
# Lets reshape test images too!
test_images = np.reshape(
  test_images,
  newshape=(test_images.shape[0], test_images.shape[1], test_images.shape[2], 1)
)

#test_images.shape
index = 100
total_imgs = 20
subset_test_images = test_images[index: index + total_imgs]
results = autoencoder.predict(
    np.reshape(subset_test_images, newshape=(total_imgs, 28, 28, 1))
)

In [ ]:
plt.figure(figsize=(15,15))
for i in range(4):
  plt.subplot(10,2,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(np.reshape(subset_test_images[i], newshape=(28, 28)))
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
for i in range(4):
  plt.subplot(10,2,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(np.reshape(results[i], newshape=(28, 28)))
plt.show()
